## Calculate ERPs

In [1]:
#Importing files and modules

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
from os import path, makedirs
#%matplotlib notebook
from brainpipe.system import study
from brainpipe.visual import *
from brainpipe.statistics import *
from mne.baseline import rescale
from mne.filter import filter_data
import pandas as pd
from pandas import ExcelWriter

from brainpipe.feat.utils._feat import _manageWindow

## User variables

In [2]:
low_pass_filter = 10.
sf = 512.
norm_mode = 'mean' #'ratio' 'mean' 'percent' 
baseline = [1408,1536] #-250ms à 0ms
data_to_use = [-250,2050] # in ms
n_perm = 200

# Compute ERPs By Odor ALL & selected trials

In [3]:
def windows(array, win, step=1):
    if step == 0: # otherwise infinite loop
        raise ValueError("Parameter 'step' can't be 0")
    lst = list(array)
    i = 0
    while i + win < len(lst):
        yield lst[i:i+win]
        i += step

def downsample(array,win,step):
    ds_data = []
    for win in windows(array, win, step):
        mean = np.mean(win)
        ds_data = np.append(ds_data,mean)
    return ds_data

def timing_ds(time,win,step,sf=512,onset=None):
    time_points = []
    for win in windows(time, win, step):
        t = ((win[int(round((len(win)/2),0))]/512)*1000)-onset
        time_points = np.append(time_points,t)
    return time_points

In [5]:
st = study('Olfacto')
path_data0 = path.join(st.path, 'database/Retrieval_No_Odor/')
path_data1 = path.join(st.path, 'database/Retrieval_EpiPerf_LowHigh/')
path2save = path.join(st.path,'feature/ERP_Odor_No_Odor/Retrieval/')
########################################################################
if not path.exists(path2save):
    makedirs(path2save)
########################################################################

subjects = ['LEFC','PIRJ','SEMC','CHAF','FERJ','VACJ']
#subjects = ['CHAF', 'VACJ', 'SEMC', 'PIRJ','MICP'] # 'LEFC',
conds = ['no_odor','low','high']

for su in subjects:
    mat0 = np.load(path_data0+su+'_odor_'+conds[0]+'_bipo_sel_phys.npz')
    data0,labels,channels = mat0['x'],mat0['labels'],mat0['channels']
    data1 = np.load(path_data1+su+'_odor_'+conds[1]+'_bipo_sel_phys.npz')['x']
    data2 = np.load(path_data1+su+'_odor_'+conds[2]+'_bipo_sel_phys.npz')['x']
    data_odor = np.concatenate((data1,data2), axis=2)
    print(su,data0.shape, data_odor.shape)
    
    cond0_erps, cond1_erps = np.array([]), np.array([])
    for elec in range(data0.shape[0]):
        fig = plt.figure()

        # Select data for one elec + name :
        odor_elec = data_odor[elec].swapaxes(0,1)
        no_odor_elec = data0[elec].swapaxes(0,1)
        channel,label = channels[elec],labels[elec]
        concat = np.concatenate((odor_elec,no_odor_elec), axis=0)
        concat = np.array(concat, dtype='float64')
        y = [0]*odor_elec.shape[0]+[1]*no_odor_elec.shape[0]
        #print(odor_elec.shape, no_odor_elec.shape, concat.shape,len(y),y)
                
        #Filter data for one elec (all trials):
        filtered_data = filter_data(concat, sfreq=512, l_freq=None, 
                    h_freq=low_pass_filter, method='fir', phase='zero-double').swapaxes(0,1)
        print ('Size of filtered data:', filtered_data.shape)

        #Normalize the non-averaged data (all trials)
        times = np.arange(filtered_data.shape[0])
        print ('time points : ', times.shape)
        filtered_data_to_norm = np.swapaxes(filtered_data, 0, 1)
        norm_filtered_data = rescale(filtered_data_to_norm, times=times, baseline=baseline, mode=norm_mode)
        norm_filtered_data = np.swapaxes(norm_filtered_data, 0, 1)
        print ('Size norm & filtered data : ', norm_filtered_data.shape,)

# =======================PREPARE DATA TO PLOT AND PLOT THE ERPs=====================================
        # Downsample each trial STER and extract time
        time = np.arange(norm_filtered_data.shape[0])
        ds_time = timing_ds(time,win=358,step=51, sf=512, onset=3000)
        time_plot = [t for t in ds_time if data_to_use[0] < t <= data_to_use[1]]
        
        ds_data_all_plot = np.array([])
        for trial in range(norm_filtered_data.shape[1]):
            ds_data = downsample(norm_filtered_data[:,trial],win=358, step=51)
            ds_plot = [p for i,p in enumerate(ds_data) if data_to_use[0] < ds_time[i] <= data_to_use[1]]
            ds_data_all_plot = np.vstack((ds_data_all_plot,ds_plot)) if np.size(ds_data_all_plot) else ds_plot
        ds_data_all_plot = ds_data_all_plot.swapaxes(0,1)
        print('-> Shape of all trials data to plot : ',ds_data_all_plot.shape)
        print('-> Shape of time vector : ', len(time_plot))

        #Plot all trials 
        fig.suptitle(su+' ERP Odor/No Odor '+str(label)+' ('+str(channel)+') ', fontsize=14)
        BorderPlot(time_plot, ds_data_all_plot, y=y, xlabel='Time (ms)',ylabel = 'ERP',
                   alpha=0.3,linewidth=2, color = ['orange','blue'],legend=['odor','no odor'])
        plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
        rmaxis(plt.gca(), ['right', 'top'])
        addLines(plt.gca(), vLines=[0], vColor=['firebrick']*2, vWidth=[2]*2, hLines=[0], hColor=['#000000'], hWidth=[2])

        plt.legend(loc=0, handletextpad=0.1, frameon=False, fontsize = 'xx-small')
        
        #Save all the plots
        fname = (path2save +su+ '_ERP_'+str(label)+'_'+str(channel)+'_('+str(elec)+').png')
        print (fname)
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.clf()
        plt.close
        
        #Save STER for each condition
        yClass = np.unique(y)
        cond0 = ds_data_all_plot[:, np.where(y == yClass[0])[0]][np.newaxis]
        cond1 = ds_data_all_plot[:, np.where(y == yClass[1])[0]][np.newaxis]
        cond0_erps = np.vstack((cond0_erps,cond0)) if np.size(cond0_erps) else cond0
        cond1_erps = np.vstack((cond1_erps,cond1)) if np.size(cond1_erps) else cond1
    
        print('data to save', ds_data_all_plot.shape,'cond0',cond0_erps.shape,'cond1',cond1_erps.shape)
        del label, elec, channel, odor_elec, no_odor_elec, concat, filtered_data, filtered_data_to_norm, 
        norm_filtered_data, times, time_plot, ds_data_all_plot
    print(cond1_erps.shape,cond0_erps.shape)
    np.save(path2save+su+'_odor_bipo_sel_phys.npy',cond0_erps)
    np.save(path2save+su+'_no_odor_bipo_sel_phys.npy',cond1_erps)

-> Olfacto loaded
LEFC (148, 3584, 129) (148, 3584, 24)
Setting up low-pass filter at 10 Hz
fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected
Size of filtered data: (3584, 153)
time points :  (3584,)
Applying baseline correction (mode: mean)
Size norm & filtered data :  (3584, 153)


<ipython-input-5-154bfa86badc>:37: DeprecationWarning: fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
  h_freq=low_pass_filter, method='fir', phase='zero-double').swapaxes(0,1)


-> Shape of all trials data to plot :  (23, 153)
-> Shape of time vector :  23
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/feature/ERP_Odor_No_Odor/Retrieval/LEFC_ERP_Amg-pPirT_a2-a1_(0).png
data to save (23, 153) cond0 (1, 23, 24) cond1 (1, 23, 129)
Setting up low-pass filter at 10 Hz
fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected
Size of filtered data: (3584, 153)
time points :  (3584,)
Applying baseline correction (mode: mean)
Size norm & filtered data :  (3584, 153)


<ipython-input-5-154bfa86badc>:37: DeprecationWarning: fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
  h_freq=low_pass_filter, method='fir', phase='zero-double').swapaxes(0,1)


-> Shape of all trials data to plot :  (23, 153)
-> Shape of time vector :  23
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/feature/ERP_Odor_No_Odor/Retrieval/LEFC_ERP_Ins-Ppo_a6-a5_(1).png
data to save (23, 153) cond0 (2, 23, 24) cond1 (2, 23, 129)
Setting up low-pass filter at 10 Hz
fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected
Size of filtered data: (3584, 153)
time points :  (3584,)
Applying baseline correction (mode: mean)
Size norm & filtered data :  (3584, 153)


...

Size of filtered data: (3584, 284)
time points :  (3584,)
Applying baseline correction (mode: mean)
Size norm & filtered data :  (3584, 284)
-> Shape of all trials data to plot :  (23, 284)
-> Shape of time vector :  23
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/feature/ERP_Odor_No_Odor/Retrieval/VACJ_ERP_STG_t8-t7_(77).png
data to save (23, 284) cond0 (78, 23, 24) cond1 (78, 23, 260)
Setting up low-pass filter at 10 Hz
fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected


<ipython-input-5-154bfa86badc>:37: DeprecationWarning: fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
  h_freq=low_pass_filter, method='fir', phase='zero-double').swapaxes(0,1)


Size of filtered data: (3584, 284)
time points :  (3584,)
Applying baseline correction (mode: mean)
Size norm & filtered data :  (3584, 284)
-> Shape of all trials data to plot :  (23, 284)
-> Shape of time vector :  23
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/feature/ERP_Odor_No_Odor/Retrieval/VACJ_ERP_STG_t9-t8_(78).png
data to save (23, 284) cond0 (79, 23, 24) cond1 (79, 23, 260)
Setting up low-pass filter at 10 Hz
fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected


<ipython-input-5-154bfa86badc>:37: DeprecationWarning: fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
  h_freq=low_pass_filter, method='fir', phase='zero-double').swapaxes(0,1)


Size of filtered data: (3584, 284)
time points :  (3584,)
Applying baseline correction (mode: mean)
Size norm & filtered data :  (3584, 284)
-> Shape of all trials data to plot :  (23, 284)
-> Shape of time vector :  23
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/feature/ERP_Odor_No_Odor/Retrieval/VACJ_ERP_STG_t10-t9_(79).png
data to save (23, 284) cond0 (80, 23, 24) cond1 (80, 23, 260)
(80, 23, 260) (80, 23, 24)
